## Introduction

We reconsider examples 6.2, 6.3, and 6.4 from 

>R.B. Sidje, [Expokit: A software package for computing matrix exponentials](http://www.maths.uq.edu.au/expokit/paper.pdf), ACM Trans. Math. Softw., 24(1):130-156, 1998. 

For these examples the Julia package [HarwellRutherfordBoeing.jl](https://github.com/JuliaSparse/HarwellRutherfordBoeing.jl) has to be
installed:

In [1]:
Pkg.add("HarwellRutherfordBoeing")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of HarwellRutherfordBoeing
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
using Expokit
using HarwellRutherfordBoeing

## Example 6.2 

Construct Hermitian matrix with the same sparsity pattern as  the one stored in `data/bcspwr10`. Both real and imaginary parts are filled using uniform random numbers within the range [-5,+5].

In [3]:
A = HarwellBoeingMatrix("data/bcspwr10").matrix
I,J,V = findnz(A)
A = sparse(I,J,V+0im)
A.nzval = rand(length(A.nzval))*10.0-5.0 + 1im*(rand(length(A.nzval))*10.0-5.0)
A = A+A'-spdiagm(real(diag(A)))

5300x5300 sparse matrix with 21842 Complex{Float64} entries:
	[1   ,    1]  =  0.91253+0.0im
	[1245,    1]  =  3.66503-3.72877im
	[2319,    1]  =  -4.10521+1.84511im
	[4939,    1]  =  4.6657+2.27286im
	[2   ,    2]  =  1.10936+0.0im
	[3036,    2]  =  3.55768-3.28849im
	[3   ,    3]  =  -2.80165+0.0im
	[2085,    3]  =  2.42611+1.59611im
	[4   ,    4]  =  4.65558+0.0im
	[238 ,    4]  =  -3.88009-1.16195im
	⋮
	[1551, 5299]  =  -4.92907-4.2527im
	[3187, 5299]  =  4.92063-3.92145im
	[4246, 5299]  =  -2.39662+3.21345im
	[5200, 5299]  =  4.89391+1.53221im
	[5299, 5299]  =  -3.34148+0.0im
	[401 , 5300]  =  -4.24611-0.683477im
	[638 , 5300]  =  1.5525-4.27671im
	[2061, 5300]  =  -3.97319+3.29075im
	[4506, 5300]  =  -2.65486-4.91032im
	[4898, 5300]  =  1.54405+1.67126im
	[5300, 5300]  =  -0.60793+0.0im

Construct $v=(1,0,\dots,0,1)^T$:

In [4]:
v = zeros(A.n)
v[1] = 1
v[A.n] = 1
v

5300-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0

First, we use the method for general complex matrices (i.e., we don't utilze the fact that A is hermitian).
We do the calculation twice, where only the second is timed, because for the first the function has to be compiled which takes additional time.

In [5]:
w = expv(1, A, v, m=30, tol=1e-5)
@time w = expv(1, A, v, m=30, tol=1e-5)
@time w = expv(1, A, v, m=30, tol=1e-5)

  0.027512 seconds (1.29 k allocations: 10.460 MB)
  0.031685 seconds (1.14 k allocations: 10.419 MB, 15.58% gc time)


5300-element Array{Complex{Float64},1}:
             1.20316e6+142.245im   
  -1.12569e-9+1.16142e-9im         
               7.96015-4.17107im   
  0.000119956-4.56964e-5im         
   4.01254e-7-9.29358e-7im         
              0.606195-1.559im     
 -0.000247977+1.63482e-5im         
              -5.24601+2.62576im   
                1.0001+0.00963894im
              -2.29763-0.308523im  
  3.31307e-11+1.47658e-12im        
  -4.09811e-5+8.07347e-5im         
              0.130963-0.340593im  
                      ⋮            
              -301.893+224.173im   
              -11016.7-62007.8im   
              -40262.6-46273.9im   
              -93077.4+58847.4im   
              -4866.28+1561.76im   
               952.458-871.216im   
               375.195-2661.45im   
               0.36865-0.673181im  
            -0.0158491+0.0143212im 
  -8.14181e-5-9.88882e-6im         
              -29004.3+32814.9im   
             2.96419e5-142.245im   

Now we use the method for hermitian complex matrices:

In [6]:
w = expv(1, A, v, m=30, tol=1e-5, hermitian=true)
@time w = expv(1, A, v, m=30, tol=1e-5, hermitian=true)
@time w = expv(1, A, v, m=30, tol=1e-5, hermitian=true)

  0.016030 seconds (1.14 k allocations: 10.419 MB)
  0.017533 seconds (1.14 k allocations: 10.419 MB, 10.36% gc time)


5300-element Array{Complex{Float64},1}:
             1.20316e6+142.245im   
  -1.12569e-9+1.16142e-9im         
               7.96015-4.17107im   
  0.000119956-4.56964e-5im         
   4.01254e-7-9.29358e-7im         
              0.606195-1.559im     
 -0.000247977+1.63482e-5im         
              -5.24601+2.62576im   
                1.0001+0.00963894im
              -2.29763-0.308523im  
  3.31307e-11+1.47658e-12im        
  -4.09811e-5+8.07347e-5im         
              0.130963-0.340593im  
                      ⋮            
              -301.893+224.173im   
              -11016.7-62007.8im   
              -40262.6-46273.9im   
              -93077.4+58847.4im   
              -4866.28+1561.76im   
               952.458-871.216im   
               375.195-2661.45im   
               0.36865-0.673181im  
            -0.0158491+0.0143212im 
  -8.14181e-5-9.88882e-6im         
              -29004.3+32814.9im   
             2.96419e5-142.245im   

## Example 6.3